# Wasserstein Deep Convolutional Conditional Generative Adversarial Network (with Gradient Penalty) (WDCCGAN-GP)

A model skeleton for Wasserstein Deep Convolutional Generative Adversarial Networks (with Gradient Penalty) (WDCCGAN-GP)s. It was first introduced in the paper titled *[Improved Training of Wasserstein GANs](https://arxiv.org/abs/1704.00028)* by Ishaan Gulrajani, Faruk Ahmed, Martin Arjovsky, Vincent Dumoulin, Aaron Courville.

## usage

In [ ]:
import deeply
import deeply.datasets as dd
from deeply.transformers.scaler import image_scaler

mnist, info = dd.load("mnist", shuffle_files = True, as_supervised = True, with_info = True)
image_shape = info.features["image"].shape
n_classes   = info.features["label"].num_classes

gan = deeply.hub("wdcgan-gp", input_shape = image_shape, n_classes = n_classes,
                 decoder_batch_norm = True, encoder_dropout_rate = 0.3,
                 encoder_layer_growth_rate = 2, scaler = image_scaler,
                 init_decoder_units = 256, decoder_layer_growth_rate = 0.5, kernel_size = 5,
                 decoder_strides  = [1, 2], final_activation = "tanh")

## example

In [ ]:
import tensorflow as tf
from deeply.plots import imgplot as imgplt, mplt

from tqdm.auto import tqdm, trange

def mapper(image, label):
    return image_scaler.fit_transform(image), tf.one_hot(label, n_classes)

config = dict(batch_size = 256,
              epochs = 20)
batch_size = config["batch_size"]

data   = mnist["train"].map(mapper)
batch  = data.batch(batch_size)

In [ ]:
from deeply.model.gan import save_img_samples

samples = gan.get_random_latent_vector(shape = (batch_size, None))

for e in trange(config["epochs"]):
    for b in tqdm(batch):
        history = gan.fit(b, batch_size = batch_size, checkpoint_path = "gan.h5d5")
    save_img_samples(gan, samples = samples, to_file = "cgan.png")